A Data Science Framework: To Achieve 99% Accuracy
    
https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy

https://medium.com/micin-developers/decipher-kernel-titanic-freeman-b48e069f76e8

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
pandas version: 0.22.0
matplotlib version: 2.1.2
NumPy version: 1.15.2
SciPy version: 1.0.0
IPython version: 7.4.0
scikit-learn version: 0.19.1
-------------------------


In [16]:
from subprocess import check_output
print(check_output(["dir",''], shell=True).decode("cp932"))

# Macならこう
# print(check_output(["ls", "../input"]).decode("utf8"))

 ドライブ C のボリューム ラベルは Windows です
 ボリューム シリアル番号は C0EA-3C28 です

 C:\Users\iidatakuya\jupyter_program\kaggle_practice\titanic のディレクトリ

2019/09/02  03:20    <DIR>          .
2019/09/02  03:20    <DIR>          ..
2019/09/02  02:35    <DIR>          .ipynb_checkpoints
2019/09/02  02:13            28,008 001_titanic.ipynb
2019/09/02  03:20             9,339 002_titanic_A_Data_Science_Framework_To_Achieve_99%_Accuracy.ipynb
2019/09/02  02:11             3,258 gender_submission.csv
2018/04/09  05:33            28,629 test.csv
2018/04/09  05:33            61,194 train.csv
               5 個のファイル             130,428 バイト
               3 個のディレクトリ  12,562,268,160 バイトの空き領域



# Step 1: Define the Problem（課題設定をする）  
- 課題：「Titanic沈没時の生存結果を予測するアルゴリズムの開発」


# Step 2: Gather the Data（データ収集）
- データ収集方法：KaggleからDownloadする。
   
  
KaggleではStep1,2は済んでいるので、3から行う


# Step 3: Prepare Data for Consumption（前処理を行う）  
- 3.1 Import Libraries（必要なライブラリのインポート）
- 3.2 Meet and Greet Data (データとの交流)
- 3.21 The 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting
       (Correcting：異常値と外れ値への対策, Completing：欠損値の補完, Creating：分析のための新特徴生成, Converting：機械学習の入力とするための数値化)
- 3.22 Clean Data（データの整形）

In [25]:
# 3.1 Import Libraries


#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

# xgboostないときは
# !pip install xgboostすればおｋ

In [24]:
# 3.2 Meet and Greet Data


#import data from file: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
data_raw = pd.read_csv('train.csv') # raw：未加工の、生の


#a dataset should be broken into 3 splits: train, test, and (final) validation
#the test file provided is the validation file for competition submission
#we will split the train set into train and test data in future sections
data_val  = pd.read_csv('test.csv') # val：評価用の


#to play with our data we'll create a copy
#remember python assignment or equal passes by reference vs values, so we use the copy function: https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs
data1 = data_raw.copy(deep = True) # defaultはtrueなのでdeep=Trueを省略してcopy()で良い 

#however passing by reference is convenient, because we can clean both datasets at once
data_cleaner = [data1, data_val] # passing by reference ：参照渡し


#preview data
print (data_raw.info()) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html
#data_raw.head() #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html
#data_raw.tail() #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.tail.html
data_raw.sample(10) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
860,861,0,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,NaN,S
215,216,1,1,"Newell, Miss. Madeleine",female,31.0,1,0,35273,113.2750,D36,C
498,499,0,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0,1,2,113781,151.5500,C22 C26,S
695,696,0,2,"Chapman, Mr. Charles Henry",male,52.0,0,0,248731,13.5000,NaN,S
680,681,0,3,"Peters, Miss. Katie",female,NaN,0,0,330935,8.1375,NaN,Q
614,615,0,3,"Brocklebank, Mr. William Alfred",male,35.0,0,0,364512,8.0500,NaN,S
18,19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",female,31.0,1,0,345763,18.0000,NaN,S
572,573,1,1,"Flynn, Mr. John Irwin (""Irving"")",male,36.0,0,0,PC 17474,26.3875,E25,S
426,427,1,2,"Clarke, Mrs. Charles V (Ada Maria Winfield)",female,28.0,1,0,2003,26.0000,NaN,S
559,560,1,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,36.0,1,0,345572,17.4000,NaN,S


In [27]:
# このKernelでは「1. Correcting：異常値と外れ値への対策」は行っていない。
# Freemanさんは、「2. Completing：欠損値の補完」を行うに当たり、
# 先ず、以下のコードでその様相を見える化している。


print('Train columns with null values:\n', data1.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum())
print("-"*10)

data_raw.describe(include = 'all')

Train columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------
Test/Validation columns with null values:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
----------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Ohman, Miss. Velin",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [28]:
###COMPLETING: complete or delete missing values in train and test/validation dataset
# （欠損値の補完、または欠損のあるデータ行削除を、学習データ・テストデータ・提出用の検証データに対して実施する）

for dataset in data_cleaner:    
    #complete missing age with median : Ageの欠損を、中央値で補完
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)

    #complete embarked with mode : Embarkedの欠損を、最頻値で補完
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

    #complete missing fare with median : Fareの欠損を、中央値で補完
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
#delete the cabin feature/column and others previously stated to exclude in train dataset
# 不要なカラムは削除する
drop_column = ['PassengerId','Cabin', 'Ticket']
data1.drop(drop_column, axis=1, inplace = True)
    # axis=1で列
    # inplace=Trueにすれば変更を元のデータに反映させることが可能 このとき、返り値はない。

print(data1.isnull().sum())
print("-"*10)
print(data_val.isnull().sum())

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
----------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [87]:
###CREATE: Feature Engineering for train and test/validation dataset
# （精度向上に向けた特徴設計を、学習データ・テストデータ・提出用の検証データに対して実施する）


for dataset in data_cleaner:    
    #Discrete variables
    dataset['FamilySize'] = dataset ['SibSp'] + dataset['Parch'] + 1

    dataset['IsAlone'] = 1 #initialize to yes/1 is alone
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    # （即席の汚いコードで、名前から、title(Mr,Ms,Mrsなどのこと)を抜き出す） 
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
        # expand=True でデータフレームの型で返す カラムは0,1,...と自動で番号が振られる。
        #              DefaultはFalseでSeriesが返され、valueがnp.arrayの配列になる 

        
    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    # 連続値を離散化する、頻度基準 vs 値域基準）
    
    # データ処理のデータ分類の際、よく使われるのがqcut関数とcut関数です。
    # 2つの関数は主に以下のような違いを持っています。

    # qcut関数
    # 　　指定した数だけデータを等分する
    # cut関数
    # 　　指定した領域ごとにデータを分割する
    
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4) # データを4等分する。返値はそれが属する幅を返す。なのでbinという名前をつけている

    #Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)


    
#cleanup rare title names
#print(data1['Title'].value_counts())
stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/

# 「必要な最小サンプルサイズは？」とよく聞かれます。
# これはおそらく私のお気に入りの質問の1つです。off the cuff（即席）でサンプルサイズの計算を実行することはありませんが、
# 10は適切な最小のNです。多くの場合、再現性/信頼性分析のコンテキストでこの質問に答えていますが、
# 通常は「10、10以上」と答えています。学生として、フィッティング不足を防ぐために、
# 回帰モデルには独立変数（IV）ごとに10の被験者が必要であると教えられていますが、
# このIVあたりの適切な被験者数の概念は、最小のサンプルサイズに広がるのはなぜですか？
# 判明したように、サンプルサイズの見積もりでは10（またはその周辺）が魔法の数字であると考える
# 正当な理由がいくつかあります。

# 標準の正規分布（µ = 0、sd = 1）があると仮定します。 
# Stataのいくつかの簡単なコードを使用して、95％信頼区間のスパンがサンプルサイズに対してどのように変化するかを
# 確認できます。
 
    
# 10個未満の名前はMiscとして一括で扱う
title_names = (data1['Title'].value_counts() < stat_min) #this will create a true false series with title name as index

#apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
# apply関数とlambda関数は、即席の汚いコードで、検索hitが少ない行数の値を、任意の値で置き換えることができる）
data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
print(data1['Title'].value_counts())
print("-"*10)



#preview data again
data1.info()
data_val.info()
data1.sample(10)

Mr        517
Miss      182
Mrs       125
Master     40
Misc       27
Name: Title, dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      891 non-null object
FamilySize    891 non-null int64
IsAlone       891 non-null int64
Title         891 non-null object
FareBin       891 non-null category
AgeBin        891 non-null category
dtypes: category(2), float64(2), int64(6), object(4)
memory usage: 85.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null 

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Title,FareBin,AgeBin
302,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,0.0000,S,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
708,1,1,"Cleaver, Miss. Alice",female,22.0,0,0,151.5500,S,1,1,Miss,"(31.0, 512.329]","(16.0, 32.0]"
135,0,2,"Richard, Mr. Emile",male,23.0,0,0,15.0458,C,1,1,Mr,"(14.454, 31.0]","(16.0, 32.0]"
784,0,3,"Ali, Mr. William",male,25.0,0,0,7.0500,S,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
856,1,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",female,45.0,1,1,164.8667,S,3,0,Mrs,"(31.0, 512.329]","(32.0, 48.0]"
159,0,3,"Sage, Master. Thomas Henry",male,28.0,8,2,69.5500,S,11,0,Master,"(31.0, 512.329]","(16.0, 32.0]"
426,1,2,"Clarke, Mrs. Charles V (Ada Maria Winfield)",female,28.0,1,0,26.0000,S,2,0,Mrs,"(14.454, 31.0]","(16.0, 32.0]"
131,0,3,"Coelho, Mr. Domingos Fernandeo",male,20.0,0,0,7.0500,S,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
702,0,3,"Barbara, Miss. Saiide",female,18.0,0,1,14.4542,C,2,0,Miss,"(7.91, 14.454]","(16.0, 32.0]"
410,0,3,"Sdycoff, Mr. Todor",male,28.0,0,0,7.8958,S,1,1,Mr,"(-0.001, 7.91]","(16.0, 32.0]"


In [86]:
# ------------------------実験-----------------------------------------------

print(dataset['Name'].head())
print("-"*100)
aaa= dataset['Name'].str.split(", ") # Series型
bbb= dataset['Name'].str.split(", ", expand=True) # DF型
ccc= dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
ddd = ccc.value_counts()
print(aaa.head())
print("-"*100)
print(bbb.head())
print("-"*100)
print(bbb[1].head())
print("-"*100)
print(ccc.head())
print("-"*100)
print(ddd) # Series型
eee = (ddd < 10)
print("-"*100)
print(eee)
ddd = ccc.apply(lambda x: 'Misc' if eee.loc[x] == True else x)
print("-"*100)
print(ddd.head())

0                                Kelly, Mr. James
1                Wilkes, Mrs. James (Ellen Needs)
2                       Myles, Mr. Thomas Francis
3                                Wirz, Mr. Albert
4    Hirvonen, Mrs. Alexander (Helga E Lindqvist)
Name: Name, dtype: object
----------------------------------------------------------------------------------------------------
0                                [Kelly, Mr. James]
1                [Wilkes, Mrs. James (Ellen Needs)]
2                       [Myles, Mr. Thomas Francis]
3                                [Wirz, Mr. Albert]
4    [Hirvonen, Mrs. Alexander (Helga E Lindqvist)]
Name: Name, dtype: object
----------------------------------------------------------------------------------------------------
          0                                   1
0     Kelly                           Mr. James
1    Wilkes            Mrs. James (Ellen Needs)
2     Myles                  Mr. Thomas Francis
3      Wirz                          Mr. Alb

In [66]:
# ------------------------実験-----------------------------------------------
print(pd.qcut(dataset['Fare'], 4).head())
print("-"*100)
print(pd.cut(dataset['Age'].astype(int), 5).head()) # Ageがなぜかfloat型なのでintに変えている


0    (-0.001, 7.896]
1    (-0.001, 7.896]
2    (7.896, 14.454]
3    (7.896, 14.454]
4    (7.896, 14.454]
Name: Fare, dtype: category
Categories (4, interval[float64]): [(-0.001, 7.896] < (7.896, 14.454] < (14.454, 31.472] < (31.472, 512.329]]
----------------------------------------------------------------------------------------------------
0    (30.4, 45.6]
1    (45.6, 60.8]
2    (60.8, 76.0]
3    (15.2, 30.4]
4    (15.2, 30.4]
Name: Age, dtype: category
Categories (5, interval[float64]): [(-0.076, 15.2] < (15.2, 30.4] < (30.4, 45.6] < (45.6, 60.8] < (60.8, 76.0]]


In [102]:
# ------------------------実験-----------------------------------------------
from pandas import Series # データ列 
from pandas import DataFrame # データフレーム
from numpy.random import randn
a = DataFrame(randn(9).reshape((3,3)),index=['A','B','D'],columns=['col1','col2','col3'])
b = DataFrame(randn(9).reshape((3,3)),index=['A','B','D'],columns=['col1','col2','col3'])
c = [a,b*2]
print(c)
for sample in c:
    continue
print(sample) # forで生成した個別のイテラブルデータの変数は消えずに最後の要素が代入されたまま残る

[       col1      col2      col3
A -0.156681 -0.188938 -1.305385
B -2.320031 -1.199571 -0.088003
D -0.020206  0.209203 -0.100704,        col1      col2      col3
A  0.437005  1.848391  2.517346
B  2.453696  1.718047 -0.294165
D -1.705802  1.507832 -2.854228]
       col1      col2      col3
A  0.437005  1.848391  2.517346
B  2.453696  1.718047 -0.294165
D -1.705802  1.507832 -2.854228


In [103]:
#CONVERT: convert objects to category using Label Encoder for train and test/validation dataset
#（カテゴリ分類にコード値が用いられている変数について、1-K符号化や、コードから連続値への変換を、学習データ・テストデータ・提出用の検証データに対して実施する）


#code categorical data
# 文字列などを0,1,2...というダミー変数に置き換える
label = LabelEncoder()
for dataset in data_cleaner:    
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])
    dataset['Title_Code'] = label.fit_transform(dataset['Title'])
    dataset['AgeBin_Code'] = label.fit_transform(dataset['AgeBin'])
    dataset['FareBin_Code'] = label.fit_transform(dataset['FareBin'])


#define y variable aka target/outcome
Target = ['Survived']

#define x variables for original features aka feature selection
data1_x = ['Sex','Pclass', 'Embarked', 'Title','SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone'] #pretty name/values for charts
data1_x_calc = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code','SibSp', 'Parch', 'Age', 'Fare'] #coded for algorithm calculation
data1_xy =  Target + data1_x
print('Original X Y: ', data1_xy, '\n')


#define x variables for original w/bin features to remove continuous variables
data1_x_bin = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code']
data1_xy_bin = Target + data1_x_bin
print('Bin X Y: ', data1_xy_bin, '\n')


#define x and y variables for dummy features original
data1_dummy = pd.get_dummies(data1[data1_x])
data1_x_dummy = data1_dummy.columns.tolist()
data1_xy_dummy = Target + data1_x_dummy
print('Dummy X Y: ', data1_xy_dummy, '\n')



data1_dummy.head()

Original X Y:  ['Survived', 'Sex', 'Pclass', 'Embarked', 'Title', 'SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone'] 

Bin X Y:  ['Survived', 'Sex_Code', 'Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code'] 

Dummy X Y:  ['Survived', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master', 'Title_Misc', 'Title_Miss', 'Title_Mr', 'Title_Mrs'] 



,Pclass,SibSp,Parch,Age,Fare,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Misc,Title_Miss,Title_Mr,Title_Mrs
0,3,1,0,22.0,7.2500,2,0,0,1,0,0,1,0,0,0,1,0
1,1,1,0,38.0,71.2833,2,0,1,0,1,0,0,0,0,0,0,1
2,3,0,0,26.0,7.9250,1,1,1,0,0,0,1,0,0,1,0,0
3,1,1,0,35.0,53.1000,2,0,1,0,0,0,1,0,0,0,0,1
4,3,0,0,35.0,8.0500,1,1,0,1,0,0,1,0,0,0,1,0


In [105]:
print('Train columns with null values: \n', data1.isnull().sum())
print("-"*10)
print (data1.info())
print("-"*10)

print('Test/Validation columns with null values: \n', data_val.isnull().sum())
print("-"*10)
print (data_val.info())
print("-"*10)

data_raw.describe(include = 'all')


# これにて、4CによるData Cleaningが完了しました。
# そして、このChapterの最後に、この後の機械学習実施のために、学習データ・テストデータに分割しています。

Train columns with null values: 
 Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         0
FamilySize       0
IsAlone          0
Title            0
FareBin          0
AgeBin           0
Sex_Code         0
Embarked_Code    0
Title_Code       0
AgeBin_Code      0
FareBin_Code     0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 19 columns):
Survived         891 non-null int64
Pclass           891 non-null int64
Name             891 non-null object
Sex              891 non-null object
Age              891 non-null float64
SibSp            891 non-null int64
Parch            891 non-null int64
Fare             891 non-null float64
Embarked         891 non-null object
FamilySize       891 non-null int64
IsAlone          891 non-null int64
Title            891 non-null object
FareBin          891 non-null catego

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Ohman, Miss. Velin",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [106]:
#split train and test data with function defaults
#random_state -> seed or control random number generator: https://www.quora.com/What-is-seed-in-random-number-generation
train1_x, test1_x, train1_y, test1_y = model_selection.train_test_split(data1[data1_x_calc], data1[Target], random_state = 0)
train1_x_bin, test1_x_bin, train1_y_bin, test1_y_bin = model_selection.train_test_split(data1[data1_x_bin], data1[Target] , random_state = 0)
train1_x_dummy, test1_x_dummy, train1_y_dummy, test1_y_dummy = model_selection.train_test_split(data1_dummy[data1_x_dummy], data1[Target], random_state = 0)


print("Data1 Shape: {}".format(data1.shape))
print("Train1 Shape: {}".format(train1_x.shape))
print("Test1 Shape: {}".format(test1_x.shape))

train1_x_bin.head()

Data1 Shape: (891, 19)
Train1 Shape: (668, 8)
Test1 Shape: (223, 8)


,Sex_Code,Pclass,Embarked_Code,Title_Code,FamilySize,AgeBin_Code,FareBin_Code
105,1,3,2,3,1,1,0
68,0,3,2,2,7,1,1
253,1,3,2,3,2,1,2
320,1,3,2,3,1,1,0
706,0,2,2,4,1,2,1
